# Проект "Рекомендация тарифов для клиентов компании Мегалайн"

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

В проекте необходимо:
- Осмотреть имеющиеся данные. Внести корректироки (если необходимо).
- Разделить исходные данные на обучающую, валидационную и тестовую выборки.
- Исследовать качество разных моделей, меняя гиперпараметры. 
- Построить модель с максимально большим значением *accuracy* (не менее 0.75). 
- Проверить *accuracy* на тестовой выборке.
- Сделать общий вывод.

## Загрузка и обзор данных

In [1]:
# импортируем нужные библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
# изучаем имеющиеся данные
data = pd.read_csv('C:/Users/user/Yandex Projects/users_behavior.csv')
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


1. Мы имеем массив данных пользовтелей, которые уже находятся на тарифе Смарт или Ультра. За определение, какой из этих двух тарифов у пользователя отвечает столбец is_ultra, 1 - тариф "Ультра", 0 - тариф "Смарт". Это и есть наш целевой признак, по которому мы будем обучать модели.  
2. В нашем датасете имеется 5 колонок и 3214 строк. Пропуски во всех столбцах отсутствуют.  
3. Названия столбцов адекватны и используют snake_case. Корректировок не требуется.
4. Скорректируем типы данных столбцов, где это применимо. Столбцы calls и messages не могут иметь дробных значений, а потом могут быть переведены в int. Остальные типы данных корректны.

In [3]:
data[['calls', 'messages']] = data[['calls', 'messages']].astype(int)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int32(2), int64(1)
memory usage: 100.6 KB


In [4]:
data

,calls,minutes,messages,mb_used,is_ultra
0,40,311.90,83,19915.42,0
1,85,516.75,56,22696.96,0
2,77,467.66,86,21060.45,0
3,106,745.53,81,8437.39,1
4,66,418.74,1,14502.75,0
...,...,...,...,...,...
3209,122,910.98,20,35124.90,1
3210,25,190.36,0,3275.61,0
3211,97,634.44,70,13974.06,0
3212,64,462.32,90,31239.78,0


Данные уже были предобработаны ранее, поэтому этап подготовки данных завершен. Можно переходить к дальнейшим этапам.

## Разбивка данных на выборки

На данном этапе нам необходимо разбить все имеющиеся данные на три группы - обучающую, валидационную и тестовую.  
На первой будет обучаться наша модель, на второй мы будем валидимровать методики и подбирать гиперпараметры, а на третьей будем проверять валидированную модель.  

Все данные будут разбиты в соотношении 3:1:1 для обучающей, валидационной и тестовой выборок соответственно.  


In [5]:
# прежде чем разбивать на различные выборки, выделим отдельно признаки, на которых мы будем обучать модели и целевой признак
features_data = data.drop('is_ultra', axis=1)
target_data = data['is_ultra']


In [6]:
# сначала разделим полученные датасеты на тестовый и не-тестовый
features_data_not_test, features_data_test, target_data_not_test, target_data_test = train_test_split(
    features_data,
    target_data,
    test_size=0.2,
    random_state=12345
)


In [7]:
# тиеперь не-тестовые данные разделим на обучающие и валидационные 
# итоговая разбивка будет 3:1:1, как и запланировано

features_data_train, features_data_valid, target_data_train, target_data_valid = train_test_split(
    features_data_not_test,
    target_data_not_test,
    test_size=0.25,
    random_state=12345
)

display(features_data_train['calls'].count())
display(features_data_valid['calls'].count())
display(features_data_test['calls'].count())

1928

643

643

Итак, мы разбили все наши данные на три части в соотношении 3:1:1.  

На первой части мы будем обучать наши модели, на второй мы будем валидировать методики и подбирать гиперпараметры, а на третьей -  проверять валидированную модель.  

## Исследование моделей

На данном этапе нам необходимо изучить и сравнить между собой несколько видов моделей, а также подобрать оптимальные для каждый модели гиперпараметры.  

Цель - получить модель, итоговая точность которой составляет не менее 75% верных предсказаний.

Так как в данном проекте нам предстоит предсказывать категориальный признак (тарифы Смарт либо Ультра), будем использовать и сравнивать между собой три модели:
- Дерево решений;
- Случайный лес;
- Логистическая регрессия.

Начнём по порядку.

In [8]:
# начнём изучение моделей с дерева решений
# исследуем, на какой максимальной "глубине" древа мы можем получить наибольшую точность ответов


for depth in range(1, 26):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_data_train, target_data_train)
    
    accuracy = model.score(features_data_valid, target_data_valid)
    print(f'max_depth = {depth}, accuracy = {accuracy}')
    

max_depth = 1, accuracy = 0.7387247278382582
max_depth = 2, accuracy = 0.7573872472783826
max_depth = 3, accuracy = 0.7651632970451011
max_depth = 4, accuracy = 0.7636080870917574
max_depth = 5, accuracy = 0.7589424572317263
max_depth = 6, accuracy = 0.7573872472783826
max_depth = 7, accuracy = 0.7744945567651633
max_depth = 8, accuracy = 0.7667185069984448
max_depth = 9, accuracy = 0.7620528771384136
max_depth = 10, accuracy = 0.7713841368584758
max_depth = 11, accuracy = 0.7589424572317263
max_depth = 12, accuracy = 0.7558320373250389
max_depth = 13, accuracy = 0.749611197511664
max_depth = 14, accuracy = 0.7573872472783826
max_depth = 15, accuracy = 0.7527216174183515
max_depth = 16, accuracy = 0.749611197511664
max_depth = 17, accuracy = 0.7387247278382582
max_depth = 18, accuracy = 0.7418351477449455
max_depth = 19, accuracy = 0.7356143079315708
max_depth = 20, accuracy = 0.7293934681181959
max_depth = 21, accuracy = 0.7325038880248833
max_depth = 22, accuracy = 0.71850699844479
m

Исходя из полученных данных, мы видим, что наибольшая точность ответов достигается в случае, когда значение гиперпараметра max_depth = 7.  
Точность полученных результатов в этом случае составляет 77,4%, что укладывается в наш целевой показатель.

Но сама модель дерева решений является наименее точной среди выбранных трёх, поэтому переходим к следующей модели - случайному лесу.

In [9]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0

for est in tqdm(range(10, 101, 10)):
    for depth in range(1, 20):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_data_train, target_data_train)
        
        accuracy = model.score(features_data_valid, target_data_valid)
        
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
        
print(f'n_estimators = {best_est}, max_depth = {best_depth}, accuracy = {best_accuracy}')
        
        

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.81s/it]

n_estimators = 20, max_depth = 15, accuracy = 0.80248833592535


Итого мы получили, что наибольшую точность показывает модель с 20 деревьями и максимальной глубиной = 15. Но при данной проверке мы использовали довольно большой шаг в количестве деревьев - 10. Попробуем провести повторную проверку вблизи 20.

In [10]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0

for est in tqdm(range(10, 31)):
    for depth in range(1, 20):
        model = RandomForestClassifier(max_depth=depth, n_estimators=est, random_state=12345)
        model.fit(features_data_train, target_data_train)
        
        accuracy = model.score(features_data_valid, target_data_valid)
        
        if accuracy > best_accuracy:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
        
print(f'n_estimators = {best_est}, max_depth = {best_depth}, accuracy = {best_accuracy}')

100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:14<00:00,  1.46it/s]

n_estimators = 20, max_depth = 15, accuracy = 0.80248833592535


Результат неизменен. Наилучший показатель точности предсказания в 80,2% имеет модель случайного леса с количеством деревьев 20 и максимальной глубиной 15.

Теперь перейдем к исследованию последнего типа модели - логистической регрессии.

In [11]:
model = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=12345)
model.fit(features_data_train, target_data_train)
    
accuracy = model.score(features_data_valid, target_data_valid)
accuracy

0.7262830482115086

Видим, что для метода логистической регрессии получаемый уровень точности ниже целевого - всего 72,6%.

Итого получаем, что из трёх рассмотренных моделей наивысшую точность демонстрирует модель случайного леса - 80,2%.  
На втором месте - дерево решений - 77,4%.  
И на третьем - логистическая регрессия - 72,6%.

## Проверка лучшей модели на тестовой выборке

Теперь протестируем модель, показавшую наилучший результат точности, на тестовой выборке - модель случайного леса. Будем сразу использовать значения гиперпараметров, полученные ранее.


In [12]:
# модель случайного леса
model = RandomForestClassifier(max_depth=best_depth, n_estimators=best_est, random_state=12345)
model.fit(features_data_train, target_data_train)
        
accuracy = model.score(features_data_test, target_data_test)
accuracy

0.7853810264385692

Как мы видим, модель на тестовой выборке подтвердила прохождение целевого уровня точности в 75%.  
При этом модель случайного леса немного ухудшила свой показатель точности.
- Случайный лес на валидационной выборке (80,2%) и на тестовой выборке (78,5%).

Тем не менее, мы можем утверждать, что модель прошла валидацию и тестировку успешно.

## Проверка модели на адекватность

Сравним предсказание нашей модели с предсказанием константного ответа (столбца, полностью состоящего из нулей).

In [13]:
# объявим такой столбец
null_column = pd.Series(0, index=range(features_data_test.shape[0]))
null_column.unique()

array([0], dtype=int64)

In [14]:
model = RandomForestClassifier(max_depth=best_depth, n_estimators=best_est, random_state=12345)
model.fit(features_data_train, target_data_train)
        
accuracy_model = model.score(features_data_test, target_data_test)
accuracy_model


0.7853810264385692

In [15]:
accuracy = accuracy_score(target_data_test, null_column)
accuracy

0.6951788491446346

По результатам проведенного сравнения точности модели и столбца с константой получили, что точность модели 78,5%, а наивного столбца - 77,6%.

## Итоговый вывод

В данном проекте мы разрабатывали и сравнивали различные предиктивные модели для предложения клиентам оператора "Мегалайн" наиболее подходящего тарифа - "Смарт" или "Ультра".

Были изучены имеющиеся данные о текущих пользователях сервиса. Предобработка данных не проводилась в виду отсутствия в датасете пропусков. Были скорректированы типы столбцов для учета звонков и смс-сообщений на int.

Исходная выборка данных была разделена на три - тренировочную, валидационную и тестировочную в соотношении 3:1:1.  
На первой обучались выбранные модели, на второй подбирались гиперпараметры для максимального уровня точности, а третья использовалась для финального тестирования.

В результате исследования моделей мы получили, что из трёх рассмотренных моделей наивысшую точность демонстрируют модели: 
- Случайный лес - 80,2%.  
- Дерево решений - 77,4%.  
- Логистическая регрессия - 72,6%.

Модель, показавшая наибольший уровень точности - случайный лес. Именно эту модель мы и будем использовать для проверки на тестовой выборке.

Как мы видим, модель на тестовой выборке подтвердила прохождение целевого уровня точности в 75%.  
При этом модель случайного леса немного ухудшила свой показатель точности.
- Случайный лес на валидационной выборке (80,2%) и на тестовой выборке (78,5%).

Тем не менее, мы можем утверждать, что модель прошла валидацию и тестировку успешно.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
